In [88]:
import pandas as pd
from pandas_datareader import data
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint, uniform
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [5]:
df = pd.read_excel(r"C:\Users\saqli\Desktop\Df.xlsx")

In [6]:
df.head()

,Unnamed: 0,Treatment,Days,Alginate,Guargum,Pectin,Weightloss,Firmness,TSS,pH,L,a,b,delta_E,TPC,DPPH,inhibition
0,NaN,0,2,0.0,0.0,0.0,18.19776,468.69,4.6,4.11,14.51,4.14,19.10,9.398904,816.88,0.108,82.32
1,NaN,1,2,3.0,0.0,0.0,18.43008,457.32,5.9,4.07,14.79,7.66,19.24,7.734145,811.88,0.128,79.05
2,NaN,2,2,0.0,3.0,0.0,18.33792,513.90,5.8,4.08,20.34,14.48,21.83,7.604591,436.88,0.056,90.84
3,NaN,3,2,0.0,0.0,3.0,18.47232,591.24,6.1,4.14,17.86,11.75,18.80,5.112592,338.13,0.051,91.65
4,NaN,4,2,1.5,1.5,0.0,18.60288,632.79,6.4,4.05,19.25,10.39,15.69,2.631045,413.13,0.039,93.62


In [7]:
x = df.iloc[:,2:6]
y = df['Firmness']

In [67]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state=42)

Hyperparameter Tuning

In [56]:
param_grid = {
    'max_depth': [2, 4, 6, 8, 10, 15, 20, None],
    'criterion': ['friedman_mse', 'squared_error', 'absolute_error', 'poisson'],
    'max_features': [0.2, 0.4, 0.6, 0.8, 1.0],
    'min_samples_split': [0.1, 0.2, 0.3, 0.4],
    'min_samples_leaf': [1, 2, 4, 6, 8],
    'max_leaf_nodes': [10, 20, 30, 40, 50, None],
    'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3]
}

# Create GridSearchCV object
reg = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid=param_grid,
    cv=5,                # Number of cross-validation folds
    n_jobs=-1,          # Use all CPU cores
    verbose=2           # Detailed output
)

In [54]:
reg = GridSearchCV(DecisionTreeRegressor(),param_grid=param_grid)

In [57]:
reg.fit(X_train,y_train)

Fitting 5 folds for each of 76800 candidates, totalling 384000 fits


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(), n_jobs=-1,
             param_grid={'criterion': ['friedman_mse', 'squared_error',
                                       'absolute_error', 'poisson'],
                         'max_depth': [2, 4, 6, 8, 10, 15, 20, None],
                         'max_features': [0.2, 0.4, 0.6, 0.8, 1.0],
                         'max_leaf_nodes': [10, 20, 30, 40, 50, None],
                         'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3],
                         'min_samples_leaf': [1, 2, 4, 6, 8],
                         'min_samples_split': [0.1, 0.2, 0.3, 0.4]},
             verbose=2)

In [60]:
print("Best parameters:", reg.best_params_)
print("Best score:", reg.best_score_)

Best parameters: {'criterion': 'friedman_mse', 'max_depth': None, 'max_features': 0.8, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.3, 'min_samples_leaf': 1, 'min_samples_split': 0.1}
Best score: 0.8685344842067304


In [97]:
for importance, name in sorted(zip(rt.feature_importances_, X_train.columns),reverse=True):
  print (name, importance)

Days 0.6205280907853384
Alginate 0.1662939645268433
Guargum 0.1206538875525464
Pectin 0.0925240571352719


DecisionTreeRegressor

In [106]:
param_grid_ = reg.best_params_

In [122]:
rt = DecisionTreeRegressor(random_state=42,**param_grid_)

In [123]:
rt.fit(X_train,y_train)

DecisionTreeRegressor(criterion='friedman_mse', max_features=0.8,
                      min_impurity_decrease=0.3, min_samples_split=0.1,
                      random_state=42)

In [124]:
y_pred = rt.predict(X_test)

In [125]:
r2_score(y_test,y_pred)

0.8990756856747368

In [126]:
# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Print all metrics
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'R² Score: {r2_score(y_test, y_pred):.4f}')

Mean Absolute Error (MAE): 32.5307
Mean Squared Error (MSE): 2395.1965
Root Mean Squared Error (RMSE): 48.9407
R² Score: 0.8991
